In [161]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score

In [162]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-russian")
audio_paths = ["dataset/hr_bot_clear/3a0cb44f-76ff-11ee-844c-c09bf4619c03.mp3"]

10/12/2024 14:39:32 - INFO - huggingsound.speech_recognition.model - Loading model...


In [163]:
# Загрузите данные из файла CSV
files: list[str] = [
    # "hr_bot_noise",
    "hr_bot_clear",
    # "hr_bot_synt",
    # "dataset/annotation/luga.json",
]

li = []
for filename in files:
    json_path = f"dataset/annotation/{filename}.json"
    df = pd.read_json(json_path)
    df["audio_filepath"] = df["audio_filepath"].apply(lambda x: f"dataset/{filename}/{x}")
    li.append(df)
data = pd.concat(li, axis=0, ignore_index=True)
data = data.sample(frac=1)
data.head()

,audio_filepath,id,text,label,attribute
497,dataset/hr_bot_clear/45d7148c-76ff-11ee-a96c-c...,45d7148c-76ff-11ee-a96c-c09bf4619c03,протянуть на десять вагонов,10,10
747,dataset/hr_bot_clear/42647498-76ff-11ee-81a5-c...,42647498-76ff-11ee-81a5-c09bf4619c03,протянуть на один вагон,10,1
917,dataset/hr_bot_clear/8dde64eb-76ff-11ee-bfec-c...,8dde64eb-76ff-11ee-bfec-c09bf4619c03,осадить на один вагон,4,1
89,dataset/hr_bot_clear/6c90fe62-76ff-11ee-b2b1-c...,6c90fe62-76ff-11ee-b2b1-c09bf4619c03,осадить на пять вагонов,4,5
1474,dataset/hr_bot_clear/7ed4a616-76ff-11ee-a679-c...,7ed4a616-76ff-11ee-a679-c09bf4619c03,растянуть автосцепки,9,-1


In [164]:
data = data[:100]
data["transcription"] = model.transcribe(data["audio_filepath"])

100%|██████████| 100/100 [00:43<00:00,  2.32it/s]


In [165]:
data["transcription"] = data["transcription"].apply(lambda x: x["transcription"])
data.head()

,audio_filepath,id,text,label,attribute,transcription
497,dataset/hr_bot_clear/45d7148c-76ff-11ee-a96c-c...,45d7148c-76ff-11ee-a96c-c09bf4619c03,протянуть на десять вагонов,10,10,протянуть на десять вагонов
747,dataset/hr_bot_clear/42647498-76ff-11ee-81a5-c...,42647498-76ff-11ee-81a5-c09bf4619c03,протянуть на один вагон,10,1,протянуть на один вагон
917,dataset/hr_bot_clear/8dde64eb-76ff-11ee-bfec-c...,8dde64eb-76ff-11ee-bfec-c09bf4619c03,осадить на один вагон,4,1,а садить на один вагон
89,dataset/hr_bot_clear/6c90fe62-76ff-11ee-b2b1-c...,6c90fe62-76ff-11ee-b2b1-c09bf4619c03,осадить на пять вагонов,4,5,а садить на пять вагонов
1474,dataset/hr_bot_clear/7ed4a616-76ff-11ee-a679-c...,7ed4a616-76ff-11ee-a679-c09bf4619c03,растянуть автосцепки,9,-1,растянуть автосцепки


In [166]:
import joblib
from sklearn.metrics import r2_score
from label2id import label2id
t2l_model = joblib.load("text2label.pkl")
labels = data["transcription"].apply(lambda x: t2l_model.predict([x])[0]).apply(label2id)
r2_score(labels, data["label"])

0.8062843513429431

In [167]:
import joblib
from label2id import label2id
t2a_model = joblib.load("text2attr.pkl")
attrs = data["transcription"].apply(lambda x: t2a_model.predict([x])[0])
r2_score(attrs, data["attribute"])

0.7631384523245568